In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import os
from pathlib import Path, PurePath
import csv
from scipy import stats
import math
from collections import Counter

In [ ]:
rlist = []
extractedpath = PurePath(Path(os.getcwd()).parents[0], 'mit-bih-extracted/')
records = extractedpath / 'subject_list.csv'
with open(records) as rfile: # reads in all of the subject IDs
    recordreader = csv.reader(rfile, delimiter=' ', quotechar='|')
    for row in recordreader:
        rlist.append(row[0])

In [ ]:
def classify_rr_ints(df):
    #list of types of rr_ints for each subject
    subject_types = []
    for row in df.itertuples():
        if row.rrInt < 0.85*row.rmean: 
            #if rr_ints is less than 85% of runningmean
            #label subject type as short
            subject_types.append('short')
        elif row.rrInt > 1.15*row.rmean: 
            #if rr_ints is greater than 115% of runningmean
            #label subject type as long
            subject_types.append('long')
        else:
            #label subject type as regular
            subject_types.append('regular')
    
    return subject_types

def find_proportions(int_types): # take the interval types and count the transitions/return the proportions
    StoS = 0
    StoR = 0
    StoL = 0
    RtoS = 0
    RtoR = 0
    RtoL = 0
    LtoS = 0
    LtoR = 0
    LtoL = 0
    for idx in range(len(int_types)-1):
        if int_types[idx]=='short' and int_types[idx+1]=='short':
            StoS+=1
        elif int_types[idx]=='short' and int_types[idx+1]=='regular':
            StoR+=1
        elif int_types[idx]=='short' and int_types[idx+1]=='long':
            StoL+=1
        elif int_types[idx]=='regular' and int_types[idx+1]=='short':
            RtoS+=1
        elif int_types[idx]=='regular' and int_types[idx+1]=='regular':
            RtoR+=1
        elif int_types[idx]=='regular' and int_types[idx+1]=='long':
            RtoL+=1
        elif int_types[idx]=='long' and int_types[idx+1]=='short':
            LtoS+=1
        elif int_types[idx]=='long' and int_types[idx+1]=='regular':
            LtoR+=1
        elif int_types[idx]=='long' and int_types[idx+1]=='long':
            LtoL+=1
    
    count = len(int_types)-1
    return [StoS/count, StoR/count, StoL/count, RtoS/count, RtoR/count, RtoL/count, LtoS/count, LtoR/count, LtoL/count]

def extract_rmssd(subset): # calculate the RMSSD of a subset
    rrInts = subset['rrInt'].to_numpy()
    diffs = np.diff(rrInts)
    sum_of_squares = np.sum(diffs**2)
    return np.sqrt(sum_of_squares/len(diffs))

def shannon_entropy(subset):
    # Get the frequency of each rrint classification in the data
    frequencies = Counter(subset)
    
    # Calculate the probability of each classification
    probabilities = [float(frequency) / len(subset) for frequency in frequencies.values()]
    
    # Calculate the Shannon entropy
    entropy = -sum(probability * math.log(probability, 2) for probability in probabilities)
    
    return entropy


In [ ]:
def subset_features(record, subsetpath):
    subset_list = pd.read_parquet(subsetpath / (record+'_subset_list.parquet')) # read the subset list for that subject
    feature_dict = {} # create a dictionary to store all of the features

    # looping over all of the windows
    for idx in tqdm(range(len(subset_list)), desc="Calculating features for each window"):
        window = pd.read_parquet(subsetpath / record / (record+"-"+str(idx)+".parquet")) # read the window

        # calculate the features for each individual window
        props = find_proportions(classify_rr_ints(window))
        feature_dict.setdefault('StoS', []).append(props[0])
        feature_dict.setdefault('StoR', []).append(props[1])
        feature_dict.setdefault('StoL', []).append(props[2])
        feature_dict.setdefault('RtoS', []).append(props[3])
        feature_dict.setdefault('RtoR', []).append(props[4])
        feature_dict.setdefault('RtoL', []).append(props[5])
        feature_dict.setdefault('LtoS', []).append(props[6])
        feature_dict.setdefault('LtoR', []).append(props[7])
        feature_dict.setdefault('LtoL', []).append(props[8])

        feature_dict.setdefault('std', []).append(np.std(window['rrInt']))
        feature_dict.setdefault('cov', []).append(np.std(window['rrInt'])/np.mean(window['rrInt']))
        feature_dict.setdefault('range', []).append(np.max(window['rrInt'])-np.min(window['rrInt']))
        feature_dict.setdefault('rrInt_var', []).append(window['rrInt'].var())
        feature_dict.setdefault('rmean_var', []).append(window['rmean'].var())
        feature_dict.setdefault('rmssd', []).append(extract_rmssd(window))
        feature_dict.setdefault('mad', []).append(stats.median_abs_deviation(window['rrInt']))
        feature_dict.setdefault('iqr', []).append(stats.iqr(window['rrInt']))
        feature_dict.setdefault('entropy', []).append(shannon_entropy(window))

    feature_df = pd.DataFrame(data=feature_dict) #data frame outta dict 
    return pd.concat([subset_list, feature_df], axis=1) 


In [ ]:
subsetpath = PurePath(Path(os.getcwd()).parents[0], 'mit-bih-time-subsets/')
featurespath = PurePath(Path(os.getcwd()).parents[0], 'mit-bih-time-features/')
if not os.path.exists(featurespath):
    os.mkdir(featurespath)

for record in tqdm(rlist): # calculate the features for all of the subjects
    features = subset_features(record, subsetpath)
    features.to_parquet(featurespath / (record+".parquet")) # and then write them to disk